In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
cat_train = [feature for feature in train.columns if 'cat' in feature]
cont_train =[feature for feature in train.columns if 'cont' in feature]
cat_test = [feature for feature in test.columns if 'cat' in feature]
cont_test =[feature for feature in test.columns if 'cont' in feature]

In [ ]:
i = 1
plt.figure()
fig, ax = plt.subplots(6, 2,figsize=(20, 24))
for feature in cont_train:
    plt.subplot(6, 2,i)
    sns.histplot(train[feature],color="blue", kde=True)
    plt.xlabel(feature, fontsize=9); plt.legend()
    i += 1
plt.show()

In [ ]:
i = 1
plt.figure()
fig, ax = plt.subplots(6, 2,figsize=(20, 24))
for feature in cont_test:
    plt.subplot(6, 2,i)
    sns.histplot(test[feature],color="blue", kde=True)
    plt.xlabel(feature, fontsize=9); plt.legend()
    i += 1
plt.show()

In [ ]:
train  = train.drop('id', axis = 'columns')
test  = test.drop('id', axis = 'columns')

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(train.corr(), annot=True)

High correlated features in train datasest (above 0.7):
1. cont0 - cont10
2. cont7 - cont10
3. cont0 - cont7

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(test.corr(), annot=True)

In [ ]:
sns.catplot(x="target", kind="count", data=train)

In [ ]:
categorical_cols = ['cat'+str(i) for i in range(19)]
continous_cols = ['cont'+str(i) for i in range(11)]

In [ ]:
cols=categorical_cols+continous_cols
train_objs_num = len(train)
dataset = pd.concat(objs=[train[cols], test[cols]], axis=0)
dataset_preprocessed = pd.get_dummies(dataset,columns=categorical_cols)
train_preprocessed = dataset_preprocessed[:train_objs_num]
test_preprocessed = dataset_preprocessed[train_objs_num:]

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

from lightgbm import LGBMClassifier


preds = np.zeros(test.shape[0])        
kf = StratifiedKFold(n_splits=5,random_state=48,shuffle=True)                  
auc=[]   # list contains AUC for each fold  
n=0   
for trn_idx, test_idx in kf.split(train_preprocessed,train['target']):
    X_tr,X_val=train_preprocessed.iloc[trn_idx],train_preprocessed.iloc[test_idx]
    y_tr,y_val=train['target'].iloc[trn_idx],train['target'].iloc[test_idx]
    model = LGBMClassifier() 
    model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False) 
    preds+=model.predict_proba(test_preprocessed)[:, 1]/kf.n_splits 
    auc.append(roc_auc_score(y_val, model.predict_proba(X_val)[:, 1])) 
    print(n+1,auc[n])                                                                                       
    n+=1     

In [ ]:
preds

In [ ]:
sub = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv')
sub.head()

In [ ]:
sub['target'] = preds
sub.to_csv('Subs.csv', index=False)